In [1]:
import pandas as pd
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer



In [2]:
df_SteamGames=pd.read_csv('steam_games_cleaned.csv')
df_UserReviews=pd.read_csv('user_reviews_cleaned.csv')

In [3]:
# Se filtra el DataFrame original para obtener solo las columnas relevantes
game_columns = ['item_id', 'name']

# Se crea un nuevo DataFrame llamado df_games que contiene solo las columnas especificadas en game_columns.
df_games = df_SteamGames[game_columns]

# Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
df_joined = pd.merge(df_games, df_UserReviews[['item_id', 'review']], on='item_id', how='inner')

In [8]:
#Se crea una muestra para el modelo
muestra = df_joined.head(20000)

In [9]:
# Se crea el modelo de machine learning con Scikit-Learn
tfidf = TfidfVectorizer(stop_words='english')
muestra=muestra.fillna("")

tdfid_matrix = tfidf.fit_transform(muestra['review'])
cosine_similarity = linear_kernel( tdfid_matrix, tdfid_matrix)

In [10]:
# Se crea la funcion de recomendación de 5 juegos recomendados para el juego ingresado.
def recomendacion_juego(id: int):
    if id not in muestra['item_id'].values:
        return {'mensaje': 'No existe el id del juego.'}
    
    titulo = muestra.loc[muestra['item_id'] == id, 'name'].iloc[0]
    idx = muestra[muestra['name'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = muestra['name'].iloc[sim_ind].values.tolist()
    return {'juegos recomendados': list(sim_juegos)}

In [14]:
# Ejemplo de uso del modelo
print(recomendacion_juego(385770))

{'juegos recomendados': ['Clicker Heroes', 'AdVenture Capitalist', 'Carpe Diem', 'Clicker Heroes', 'Crusaders of the Lost Idols']}


In [13]:
muestra

,item_id,name,review
0,282010,Carmageddon Max Pack,oh where do i start dos
1,70,Half-Life,if you own any half life but not this one that...
2,70,Half-Life,this is one of the greatest games i have ever ...
3,70,Half-Life,possibly the best fps game ever
4,70,Half-Life,do i honestly need to recommend this
...,...,...,...
19995,385770,Time Clickers,needs some some more guns but other than that ...
19996,385770,Time Clickers,i can not stop playing it its is so cool
19997,385770,Time Clickers,better than clicking it in one spot
19998,385770,Time Clickers,a great spin on a new genre clickers the added...
